# SPECIO

In [1]:
%%HTML
<button id="do_run_all">Start</button>
<script>
$("#do_run_all").click(
    function () {
        $("#run_all_cells").click();
    }
);
</script>

In [2]:
%%HTML
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [3]:
%matplotlib notebook
from IPython.display import HTML, display, clear_output, Javascript
import ipywidgets as widgets
import specio
import os,sys,socket
from glob import glob
import pandas as pd
import numpy as np
import re
from jupyterthemes import jtplot

jtplot.style();
display(HTML('''<style>.widget-label { min-width: 20ex !important; }</style>''')); #widget label width
display(HTML("<style>.container { width:90% !important; }</style>")); #cell width

## By night:

In [4]:
#::: on laptop (OS X)
if sys.platform == "darwin":
    dirname = '/Users/mx/Big_Data/BIG_DATA_SPECULOOS/Observing_log/'    

#::: on Cambridge servers
elif 'ra.phy.cam.ac.uk' in socket.gethostname():
    dirname = '/appcg/data2/SPECULOOS/Observing_log/'  
        
display(HTML(filename=dirname+'Observing_log.html'))

,date,telescope,field_name,filter,exposure,N_images,N_hours
0,2018-01-21,Callisto,GJ1132,I+z,10.00,868.0,2.41
1,2018-01-22,Callisto,NGTS-1,I+z,17.97,357.0,1.78
2,2018-01-26,Callisto,Sp0805-3158,I+z,50.00,559.0,7.76
3,2018-01-27,Callisto,Sp0828-1309,I+z,50.00,479.0,6.65


## By target:

In [5]:



###############################################################################
# Helper
###############################################################################
def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)


###############################################################################
# Checkboxes
###############################################################################
#::: show overview checkbox widget    
show_overview_w = widgets.Checkbox(
    value = True,
    description='Overview plots'
)


#::: show stats checkbox widget    
show_stats_w = widgets.Checkbox(
    value = False,
    description='Observing stats table'
)


#::: show ACP pointing checkbox widget    
show_ACP_pointing_w = widgets.Checkbox(
    value = False,
    description='ACP pointing plot'
)
    
    
    
    
###############################################################################
# Drowndowns
###############################################################################
#::: telescope drowndown widget
telescope_w = widgets.Dropdown(
    options=['Io','Europa','Callisto','Ganymed'],
    value='Callisto',
    description='Telescope'
)


#::: read available fields and filters for this telescope
def get_field_and_filter_w_options():
    telescope = telescope_w.value
    try:
        buf = glob( os.path.join(specio.root(telescope), '*_output.fts') )
        fields_and_filters = [ item.split('/')[-1][0:-11] for item in buf ]
        fields_and_filters = sorted_nicely(fields_and_filters)
        return fields_and_filters
    except:
        return ['No data available']


#::: fields and filters dropdown widget
field_and_filter_w = widgets.Dropdown(
    options=get_field_and_filter_w_options(),
    description='Fields and filters'
)


#::: read available obj_ids for this telescope, field and filter
def get_obj_id_w_options():
    telescope = telescope_w.value
    try:
        field_name, filter_band = field_and_filter_w.value.split('_')
        dic = specio.get(telescope, field_name, filter_band, ['OBJ_ID'], silent=True)
        obj_ids = list(dic['OBJ_ID'])
        return obj_ids
    except:
        return ['No data available']
        

#::: OBJ_IDs dropdown widget
obj_id_w = widgets.Dropdown(
    options=get_obj_id_w_options(),
    description='Object ID'
)


#::: read available obj_ids for this telescope, field and filter
def get_date_w_options():
    try:
        telescope = telescope_w.value
        field_name, filter_band = field_and_filter_w.value.split('_')
        obj_id = obj_id_w.value
        dic = specio.get(telescope, field_name, filter_band, ['STATS'], silent=True)
        #print dic
        dates = ['all'] + list(dic['STATS'][0])
        Nimages = [str(np.sum(dic['STATS'][1]))] + [ str(item) for item in dic['STATS'][1] ]
        date_Nimages = [ item0+'    ('+item1+' img)' for item0, item1 in zip(dates, Nimages)   ]
        return date_Nimages
    except:
        return ['No data available']
        

#::: date dropdown widget
date_w = widgets.Dropdown(
    options=get_date_w_options(),
    description='Date'
)


'''
def get_output_w_value():
    try:
        dic = specio.get(telescope_w.value, field_and_filter_w.value.split('_')[0], field_and_filter_w.value.split('_')[1], ['STATS'], silent=True)
        return str(len(dic['STATS'][0])) + ' nights, ' + str(np.sum(dic['STATS'][1])) + ' images'
    except:
        return 'No data available'
    

#::: stats output widget
output_w = widgets.Text(
    value=get_output_w_value(),
    placeholder='',
    description='    ',
    disabled=True
)
'''


    
###############################################################################
# Extra plots
###############################################################################
#::: show stats checkbox widget    
extra_plot_selection_w = widgets.SelectMultiple(
    #telescope = telescope_w.value
    #field_name, filter_band = field_and_filter_w.value.split('_')
    #options = specio.get_available_keys(telescope, field_name, filter_band)[3:],
    options = sorted_nicely( ['FLUXERR', 'ERROR_8', 'ERROR_9', 'ERROR_4', 'ERROR_5', 'ERROR_6', 'ERROR_7', 'ERROR_1', 'ERROR_2', 'ERROR_3',\
               'CCDX', 'CCDY', 'PEAK', 'JD', 'ERROR_12', 'ERROR_13', 'ERROR_10', \
               'FLUX', 'FLUX_6', 'FLUX_7', 'FLUX_4', 'FLUX_5', 'FLUX_2', 'FLUX_3', 'FLUX_1', 'FLUX_8', 'FLUX_9', \
               'SKYBKG', 'FLUX_10', 'FLUX_11', 'FLUX_12', 'FLUX_13', 'ERROR_11', 'AIRMASS', 'ALTITUDE', 'AMBTEMP', 'AZIMUTH', \
               'BJD-OBS', 'CCD-TEMP', 'CROWDED', 'DEC', 'DEC_MOVE', 'DEWPOINT', 'EXPOSURE', 'FILTER', 'FILTFWHM', 'FOCALLEN', \
               'FOCUSPOS', 'FOCUSTEM', 'FRAME_SN', 'FWHM', 'HA', 'HJD-OBS', 'HUMIDITY', 'IMAGETYP', 'JD-OBS', 'MINPIX', 'NBSIZE', \
               'NUMBRMS', 'OBJECT', 'PA', 'RA', 'RA_MOVE', 'RAPOS', 'RCORE', 'SEEING', 'SKY_MOVE', 'SKYLEVEL', 'SKYNOISE', 'SKYTEMP', \
               'STDCRMS', 'TELESCOP', 'THRESHOL', 'WCSCOMPL', 'WCSF_DEC', 'WCSF_NS', 'WCSF_RA', 'WCSF_RMS', 'WINDSPD', 'XBINNING', 'YBINNING', \
               'PSF_A_1', 'PSF_B_1', 'PSF_T_1', 'PSF_A_2', 'PSF_B_2', 'PSF_T_2', 'PSF_A_3', 'PSF_B_3', 'PSF_T_3', 'PSF_A_4', 'PSF_B_4', 'PSF_T_4', \
               'PSF_A_5', 'PSF_B_5', 'PSF_T_5', 'PSF_A_6', 'PSF_B_6', 'PSF_T_6', 'PSF_A_7', 'PSF_B_7', 'PSF_T_7', 'PSF_A_8', 'PSF_B_8', 'PSF_T_8', \
               'PSF_A_9', 'PSF_B_9', 'PSF_T_9', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2', 'CRPIX1', 'CRPIX2', 'CRVAL1', 'CRVAL2', 'CTYPE1', 'CTYPE2', \
               'HICOUNT', 'LOCOUNT', 'TMID', 'NSOURCES'] ),
    disabled=False,
    description='Extra plots'
)




###############################################################################
# Update
###############################################################################
#::: selection changes, update dropdown menus
def update(b):
    field_and_filter_w.options = get_field_and_filter_w_options()
    obj_id_w.options = get_obj_id_w_options()
    date_w.options = get_date_w_options()
    
    
#::: selection changes, update dropdown menus
show_overview_w.observe(update)
extra_plot_selection_w.observe(update)
show_stats_w.observe(update)
show_ACP_pointing_w.observe(update)
telescope_w.observe(update)
field_and_filter_w.observe(update)
obj_id_w.observe(update)
date_w.observe(update)



#::: combine all widgets in an hbox
hbox1 = widgets.HBox([telescope_w, field_and_filter_w, obj_id_w, date_w])
vbox2 = widgets.VBox([show_overview_w, show_stats_w, show_ACP_pointing_w])
hbox3 = widgets.HBox([extra_plot_selection_w])


#::: display all widgets
display(hbox1)
display(vbox2)
display(hbox3)

SEJveChjaGlsZHJlbj0oRHJvcGRvd24oZGVzY3JpcHRpb249dSdUZWxlc2NvcGUnLCBpbmRleD0yLCBvcHRpb25zPSgnSW8nLCAnRXVyb3BhJywgJ0NhbGxpc3RvJywgJ0dhbnltZWQnKSwgdmHigKY=


VkJveChjaGlsZHJlbj0oQ2hlY2tib3godmFsdWU9VHJ1ZSwgZGVzY3JpcHRpb249dSdPdmVydmlldyBwbG90cycpLCBDaGVja2JveCh2YWx1ZT1GYWxzZSwgZGVzY3JpcHRpb249dSdPYnNlcnbigKY=


SEJveChjaGlsZHJlbj0oU2VsZWN0TXVsdGlwbGUoZGVzY3JpcHRpb249dSdFeHRyYSBwbG90cycsIG9wdGlvbnM9KCdBSVJNQVNTJywgJ0FMVElUVURFJywgJ0FNQlRFTVAnLCAnQVpJTVVUSCfigKY=


In [30]:
###############################################################################
# Run button
###############################################################################
#::: run all cells below the run button cell
def run_all(b): 
    display(Javascript('IPython.notebook.execute_cells_below()'))

#::: run button
button = widgets.Button(description="Query")
button.style.button_color = 'lightsteelblue'
hbox4 = widgets.HBox([button])
display(hbox4)
button.on_click(run_all)

<IPython.core.display.Javascript object>

### Infos

In [31]:
def show_infos():    
    clear_output()
    try:
        telescope = telescope_w.value
        field_name, filter_band = field_and_filter_w.value.split('_')
        obj_id = obj_id_w.value
        specio.print_infos(telescope, field_name, filter_band, obj_id=obj_id)
    except:
        print("No data available")

show_infos()


Callisto Sp2331-2749 r SP000000
Gaia ID: 2331825568489673728
Nights: 9
Images: 3691
Mean exposure: 25 s
Hours observed: 25.6 h
Date range: from 2018-06-12 to 2018-06-20
Mean flux: 2081
G-mag: 13.3


### Overview plots

In [32]:
#::: run
def show_overview():
    clear_output()
    try:
        telescope = telescope_w.value
        field_name, filter_band = field_and_filter_w.value.split('_')
        obj_id = obj_id_w.value
        date = date_w.value[0:10]
        extra_keys = list( extra_plot_selection_w.value )
        if date[0:3] == 'all': date = 'all'
        if date=='all':
            fig_overview, axes_overview = specio.plot_overview(telescope, field_name, filter_band, extra_keys=extra_keys, obj_id=obj_id)
        else:   
            fig_overview, axes_overview = specio.plot_overview(telescope, field_name, filter_band, extra_keys=extra_keys, obj_id=obj_id, time_date=date)
    except:
        print("No data available")

if show_overview_w.value:
    show_overview()
else:
    clear_output()

### Observing stats table

In [33]:
#################################################################################
#::: Observing stats table
##################################################################################

'''
#::: info widget
def get_stats_info_w_value():
    try:
        return 'Observing table for ' + telescope_w.value + ', ' + field_and_filter_w.value.split('_')[0] + ', ' + field_and_filter_w.value.split('_')[1]
    except:
        return 'No data available'
    

#::: info widget
stats_info_w = widgets.Text(
    value=get_stats_info_w_value(),
    placeholder='',
    description='    ',
    disabled=True,
    layout=widgets.Layout(width='1000px')
)
'''

#::: print stats as html table
def print_stats_html_table(telescope, field_name, filter_band):
    dic = specio.get(telescope, field_name, filter_band, ['STATS'], silent=True)
    pdic = pd.DataFrame( data={'DATE':dic['STATS'][0], 'NUM_IMAGES':dic['STATS'][1]} )
    display(pdic)

    
#::: show stats
def show_stats():
    clear_output()
    #display(stats_info_w)
    #display(stats_info_button)
    try:
        telescope = telescope_w.value
        field_name, filter_band = field_and_filter_w.value.split('_')
        obj_id = obj_id_w.value
        date = date_w.value[0:10]
        if date[0:3] == 'all': date = 'all'
        print_stats_html_table(telescope, field_name, filter_band)
    except:
        pass


if show_stats_w.value:
    show_stats()
else:
    clear_output()

### ACP pointing

In [34]:
#################################################################################
#::: ACP pointing
#################################################################################

'''
#::: info widget
def get_ACP_info_w_value():
    try:
        return 'Show ACP pointing error for ' + telescope_w.value
    except:
        return 'No data available'
    

#::: info widget
ACP_info_w = widgets.Text(
    value=get_ACP_info_w_value(),
    placeholder='',
    description='    ',
    disabled=True,
    layout=widgets.Layout(width='1000px')
)
'''


#::: show ACP pointing plot
def show_ACP_pointing_plot():
    clear_output()
    #display(ACP_info_w)
    #display(ACP_info_button)
    try:
        telescope = telescope_w.value
        field_name, filter_band = field_and_filter_w.value.split('_')
        obj_id = obj_id_w.value
        date = date_w.value[0:10]
        if date[0:3] == 'all': date = 'all'
        if date=='all':
            specio.plot_ACP_pointing(telescope)
        else:
            specio.plot_ACP_pointing(telescope,time_hjd=date)
    except:
        pass
    

#::: run button
#ACP_info_button = widgets.Button(description="Show")
#ACP_info_button.on_click(show_ACP_pointing_plot)


#display(ACP_info_w)
#display(ACP_info_button)

if show_ACP_pointing_w.value:
    show_ACP_pointing_plot()
else:
    clear_output()

In [35]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};

<IPython.core.display.Javascript object>